In [ ]:
%%capture
%run 02_transaction.ipynb

In [ ]:
from eth_account.messages import encode_defunct as encode_msg
import web3 as w3; w3 = w3.Account

Create a public key `pub_key`, private key `priv_key` pair. The public key is used as an address and the private key is used for signing txs. The `priv_key` is basically your password.

In [ ]:
def keys():
    acc = w3.create()
    return acc.privateKey.hex(), acc.address

priv_key,pub_key = keys()
ph(priv_key), ph(pub_key)

In [ ]:
priv_key

## Wallet

Holds a key pair and is used for signing txs. The `nonce` keeps track of the number of txs made with this account.

In [ ]:
class Wallet:
    def __init__(self): 
        self.priv, self.pub = keys()
        self.nonce = 0
        
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig.signature.hex()
        return tx
            
    def signed_tx(self, to, value, fee):
        tx = TX(self.pub, to.pub, value, fee, self.nonce)
        return self.sign(tx)
    
    def __str__(self): return ph(self.pub)

In [ ]:
acc1 = Wallet(); print(acc1)
acc2 = Wallet(); print(acc2)

### Transaction Signing
Signing a tx generates a signature that only the private key can produce. This can be later validated very efficiently.

In [ ]:
tx        = TX(acc1.pub, acc2.pub, 12, 0.2, acc1.nonce)
tx_signed = acc1.sign(tx)

The signature is exactly what Ethereum uses as well. There are some extra informations that the signature creates that we don't need. For more infos: https://medium.com/@angellopozo/ethereum-signing-and-validating-13a2d7cb0ee3

In [ ]:
tx_signed.sig

Accounts can generate a signed tx quickly with `signed_tx`.

In [ ]:
tx = acc1.signed_tx(acc2, 7, 0.3); print(tx)

### Transaction Validation

Validates if the tx was signed by the sender.

In [ ]:
def val_sig(tx):
    if not hasattr(tx, 'sig'): return False
    m = encode_msg(bytes(tx))
    return w3.recover_message(m, signature=tx.sig) == tx.fr

In [ ]:
assert val_sig(tx_signed)

If anything in the tx is changed, like increasing the value to send, the signature should become invalid.

In [ ]:
tx_signed.value = 30
assert not val_sig(tx_signed)